# Steganografija slike

Način na koji se može sakriti tajna poruka tokom slanja tako da ne privlači pažnju napadača jeste steganografija. U kriptografiji poruke su šifrovane te često zainteresuju napadače koji pokušavaju da razbiju šifru, dok u steganografiji sakrivena poruka se ne zaključava vidljivim ključem koji se potom odvojeno šalje, pa ne postoji objekat privlačenja pogleda napadača.
<hr>
<i>Steganografija</i> je metoda sakrivanja tajne poruke u slici, zvuku, videu itd koji će biti prenosni mediji. Kako tekstualna poruka, tako se i slika može sakriti u drugoj slici i slično. Takvom steganografijom ćemo se baviti u nastavku.

Tehnika sakrivanja poruke/slike u slici-nosaču jeste ubacivanje značajnih bitova (bitova velike težine) tajne poruke/slike na mestima najnižih bitova slike-nosača, jer na taj način slika-nosač neće biti vidljivo promenjena i tajna poruka/slika neće biti vidljiva. Bitno je izabrati dobru sliku-nosač koja je puna detalja, jer male promene u pikselima koji predstavljaju oštre ivice mogu biti vidljive. Dakle, ono što će biti promenjeno jeste intenzitet RGB boja u nekim pikselima. Procenat originalne slike koji možemo menjati je mali, 10-ak procenata da promena slike ne bi bila vidljiva. 

<hr>
Prikazivanje svakog piksela je moguće čuvanjem vrednosti boja u njemu. Koristićemo RGB model boja. Svaki piksel će imati tri 8-bitne vrednosti za svaku od boja.

![rgb](rgb.png)
<hr>
<img src="tajno.png" width="500">

Kako bismo dobili male promene slike-nosača menjaćemo vrednosti njenih najnižih bitova sa bitovima najveće težine poruke ili slike koju želimo da sakrijemo. 

Svaki piksel će biti predstavljen trojkama 8-bitnih brojeva, svaki će predstavljati jačinu jedne od RGB boja. Vrednosti idu od 0 do 255.
![rgb_bit](rgb_bit.png)

Za početak sakrićemo sliku unutar druge slike:

In [2]:
%config IPCompleter.greedy=True

In [ ]:
from PIL import Image
import numpy as np

class Steganography:
    #Returns rgb values in binary representation
    get_binary_rgb = lambda r, g, b: ('{0:08b}'.format(r, 'b'), '{0:08b}'.format(g, 'b'), '{0:08b}'.format(b, 'b'))

    @staticmethod
    def encrypt(image_holder, image_secret):
        """Method that returns new image that is a result of merging secret image into 
        image that will be send """
        
        #If image size (with and hight given as a tuple) is bigger then holder image, than we want to exit
        holder_weight = image_holder.size[0]
        holder_hight = image_holder.size[1]
        secret_weight = image_secret.size[0]
        secret_hight = image_secret.size[1]
        if secret_weight > holder_weight or secret_hight > holder_hight:
            raise ValueError('Secret image is bigger than image holder!')
        
        #We can see what are RGB values of pixcels: print np.asarray(image_holder)
        image_holder_pixels = image_holder.load()
        image_secret_pixels = image_secret.load()
       
        #Output image will be the same as holder, and then we are going to change it's pixels
        output_image = Image.new(image_holder.mode, image_holder.size)
        output_image_pixels = output_image.load()
        
        for i in range(holder_weight):
            for j in range(holder_hight): 
                r_h = image_holder_pixels[i,j][0]
                g_h = image_holder_pixels[i,j][1]
                b_h = image_holder_pixels[i,j][2]
                rgb_binary_holder = Steganography.get_binary_rgb(r_h, g_h, b_h)
                
                #Check if the secret image is finished because it is smaller than holder image
                r_s = 0
                g_s = 0
                b_s = 0          
                if(secret_weight > i and secret_hight > j):
                    r_s = image_secret_pixels[i,j][0]
                    g_s = image_secret_pixels[i,j][1]
                    b_s = image_secret_pixels[i,j][2]
                rgb_binary_secret = Steganography.get_binary_rgb(r_s, g_s, b_s)
                
                #Now we have rgb binary values for holder image and secret image that we want to merge it into
                #Then we want to place secret image into holder one
                r = rgb_binary_holder[0][:4] + rgb_binary_secret[0][:4]
                g = rgb_binary_holder[1][:4] + rgb_binary_secret[1][:4]
                b = rgb_binary_holder[2][:4] + rgb_binary_secret[2][:4]
                output_image_pixels[i, j] = (int(r, 2), int(g, 2), int(b, 2))
                
        return output_image
        
        
        
        
        
        
        
        #image_holder.close()
        #image_secret.close()
        
#f __name__ == '__main__':
encrypt = Steganography.encrypt(Image.open('New_york.jpg'), Image.open('secret_photo.jpg'))       
encrypt.save('output.jpeg')